In [27]:
import tensorflow as tf
import numpy as np
from transformers import XLNetTokenizer, TFXLNetLMHeadModel

tokenizer = XLNetTokenizer.from_pretrained("hfl/chinese-xlnet-base")

All model checkpoint layers were used when initializing TFXLNetLMHeadModel.

All the layers of TFXLNetLMHeadModel were initialized from the model checkpoint at /home/yangkaixuan/project/mama/mymodel3.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetLMHeadModel for predictions without further training.


In [115]:
from transformers import TFXLNetMainLayer
a=TFXLNetMainLayer(model.config)
a.summary()

AttributeError: 'TFXLNetMainLayer' object has no attribute 'summary'

In [98]:
CUSTOMER_TOKENS = [12967, 30]
SALES_TOKENS = [4925, 30]
model = TFXLNetLMHeadModel.from_pretrained("/home/yangkaixuan/project/mama/mymodel6")
model.transformer.attn_type = 'bi'

All model checkpoint layers were used when initializing TFXLNetLMHeadModel.

All the layers of TFXLNetLMHeadModel were initialized from the model checkpoint at /home/yangkaixuan/project/mama/mymodel6.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetLMHeadModel for predictions without further training.


In [61]:
contexts = ["很高兴服务到您",
           "我是鹤壁市浚县小河镇的，我们附近有没有你们的业务员？",
           "在鹤壁市的话暂时没有分公司，现在在江苏，河北，还有山东是有分公司的，其实您问有没有分公司，也是担心后期这方面对吧？",
           ]
inputs=[]
customer_i=[1,3,4,5]
for i,text in enumerate(contexts):
    if i in customer_i:
        inputs.extend(CUSTOMER_TOKENS)
    else:
        inputs.extend(SALES_TOKENS)
    inputs.extend(tokenizer.encode(text,add_special_tokens=False))

In [84]:
contexts=['严大哥，您上个月了解了我们公司的一款重大疾病保障，我当时电话里给您讲解过，您这边还有什么不明白的吗？您这边能不能参与进来，需要具体的给您做一个系统审核，您这边提供一下资料，帮您审核一下，审核通过的话，您才可以参与进来<eop>',
'严大哥，我看到系统显示您这边昨天晚上重新留资了',
'您这边是想了解孝亲宝这款保障吗？']
inputs=[]
customer_i=[]
for i,text in enumerate(contexts):
    if i in customer_i:
        inputs.extend(CUSTOMER_TOKENS)
    else:
        inputs.extend(SALES_TOKENS)
    inputs.extend(tokenizer.encode(text,add_special_tokens=False))

In [106]:
# We show how to setup inputs to predict a next token using a bi-directional context.
inputs1 = inputs+SALES_TOKENS+tokenizer.encode("从不信任到理<mask>", add_special_tokens=True) # We will predict the masked token
input_ids=tf.convert_to_tensor(
    inputs1, dtype=None, dtype_hint=None, name=None
)
input_ids=input_ids[None,:]
perm_mask = np.zeros((1, input_ids.shape[1], input_ids.shape[1]))
perm_mask[:, :, -1] = 1.0  # Previous tokens don't see last token

target_mapping = np.zeros(
    (1, 1, input_ids.shape[1])
)  # Shape [1, 1, seq_length] => let's predict one token
target_mapping[
    0, 0, -1
] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

outputs = model(
    input_ids,
    perm_mask=tf.constant(perm_mask, dtype=tf.float32),
    target_mapping=tf.constant(target_mapping, dtype=tf.float32),
)

next_token_logits = outputs[
    0
] 
a=tf.argsort(next_token_logits,-1,direction='DESCENDING')[0,0,:150].numpy()
b=[tokenizer.decode(x) for x in a]
print('赔' in b)
b

False


['坚持',
 '一直',
 '的',
 '意义',
 '信任',
 '客户',
 '莫',
 '这',
 '我们',
 '真',
 '从',
 '很',
 '无',
 '相信',
 '真的',
 '同事',
 '我',
 '也',
 '是',
 '\\',
 '一直在',
 '了',
 '都',
 '心',
 '您',
 '初',
 '过',
 '让',
 ',',
 '长时间',
 '时间',
 '公司的',
 '很难',
 '非常',
 '很久',
 '',
 '过程',
 '程',
 '不',
 '今天',
 '保险',
 '收到',
 '买',
 '华',
 '都是',
 '这么',
 '流程',
 '我的',
 '天',
 '这段',
 '德',
 '理解',
 '最终',
 '很多',
 '辈',
 '一天',
 '这是',
 '总',
 '大',
 '自',
 '服务',
 '速度',
 '一次',
 '来',
 '小',
 '经历了',
 '不信任',
 '其实',
 '敬',
 '真实',
 '始终',
 '得到',
 '每天',
 '整个',
 '值得',
 '风险',
 '理',
 '不是',
 '几乎',
 '公司',
 '郭',
 '一',
 '昨',
 '希望',
 '看到',
 '全程',
 '如此',
 'n',
 '一位',
 '实',
 '多',
 '对',
 '我想',
 '都在',
 '信',
 '年',
 '的时间',
 '从',
 '终于',
 '遇到',
 '工作',
 '加',
 '两',
 '人',
 '来自',
 '一直是',
 '十分',
 '看',
 '做',
 '理由',
 '经历',
 '决定',
 '余',
 '给',
 '仅',
 '去',
 '有',
 '用了',
 '高效',
 '经过',
 '衷',
 '到',
 '李',
 '好',
 '简单',
 '又',
 '挺',
 '半',
 '每一个',
 '数据',
 '回',
 '了不少',
 '难',
 '第一次',
 '还是',
 '运',
 '说',
 '成',
 '现在',
 '咱',
 '自己',
 '我是',
 '一段',
 '谁',
 '每',
 '了一',
 '不得不',
 '所有的',
 '几次',
 '觉得']

In [89]:
b

[',',
 '',
 '一直',
 '我',
 '小',
 '您',
 '我',
 '从',
 '小',
 '到',
 '都',
 '这',
 '公司',
 '我们',
 '一直在',
 '来',
 '有',
 '看',
 '三',
 '信任',
 '咱',
 '的',
 '德',
 '这',
 '3',
 '上',
 '<eop>',
 '中',
 '坚持',
 '了解',
 '全程',
 '加上',
 '大',
 '我是',
 '就',
 '多次',
 '整个',
 '再',
 '走',
 '过程',
 '?',
 '。',
 '加',
 '咨询',
 '沟通',
 '大',
 '给',
 '打',
 '我的',
 '4',
 '让',
 '服务',
 '都是',
 '呢',
 '前',
 '.',
 '、',
 '工作',
 '马',
 '也',
 '和',
 '在',
 '演变',
 '开始',
 '联系',
 '深',
 '里',
 '度',
 '理',
 '讲',
 '最后',
 '之前',
 '~',
 '了',
 '买',
 '是',
 '[',
 '路',
 '相信',
 '李',
 '产品',
 '找到',
 '以来',
 '!',
 '都有',
 '一路',
 '公司的',
 '企业',
 '经过',
 '流程',
 '这个',
 '先生',
 '哪',
 '当时',
 '聊',
 '保险',
 '后',
 '你',
 '还',
 '哥',
 '选择',
 '所有的',
 '还是',
 '在',
 '发展',
 '一直是',
 '还有',
 '这个',
 '吧',
 '做',
 '然后',
 '真的',
 '沟',
 '也是',
 '其实',
 '过来',
 '1',
 '每天',
 '边',
 '天',
 '一步',
 '来说',
 '最终',
 '官方',
 '姐',
 '一',
 '都能',
 '那',
 '认识',
 '资料',
 '时间',
 '成',
 '对',
 '来看',
 '从',
 '感',
 '的过程',
 '等',
 '留下',
 '内',
 '任',
 '总',
 '找',
 '[',
 '思',
 '微',
 '总公司',
 '之间',
 '常',
 '出',
 '保障',
 '啊',
 '今天',
 '感觉',


In [107]:
tokenizer.encode('理赔')

[19, 643, 21217, 4, 3]

In [112]:
tokenizer.encode('我从不信任到理赔信任')

[7397, 192, 20647, 64, 643, 21217, 9209, 4, 3]